## Βήμα 1 - Συλλογή Εγγράφων

Για να συλλέξουμε τα έγγραφα που χρειαζόμαστε δημιουργούμε έναν web crawler χρησιμοποιώντας το πακέτο BeautifulSoup της python. Το αρχείο που περιέχει τον κώδικα του web crawler είναι το wiki_crawler.py. Επιλέγουμε 10  κείμενα από τη wikipedia που είναι σχετικά με τις θεματικές του μαθήματος, έτσι θα έχουμε κατάλληλο όγκο πληροφορίας για να αναλύσουμε και να διαχειριστούμε με τον οποίο μπορούμε να αναδείξουμε αποτελεσματικά τις λειτουργίες της μηχανής αναζήτησης που θέλουμε να υλοποιήσουμε, χωρίς όμως να είναι υπερβολικός.

Αρχικά  κάνουμε import τις απαραίτητες βιβλιοθήκες

In [13]:
import requests
from bs4 import BeautifulSoup
import json

Στη συνέχεια αρχικοποιούμε το URL από το οποίο θα γίνει υ συλλογή των εγγράφων, στην προκειμένη περίπτωση απο τη wikipedia, και τις θεματικές των άρθρων που θα συλλέξουμε.

In [14]:
BASE_URL = "https://en.wikipedia.org/wiki/"
TOPICS = ["Information_retrieval", "Natural_language_processing", "Artificial_intelligence","Data_mining", "Machine_learning", "Deep_learning", "Computer_vision", "Neural_network", "Big_data", "Data_science"]

Ορίζουμε τη συνάρτηση scrape_wikipedia_articles, η οποία ψάχνει για τα άρθρα στο URL που έχουμε ήδη ορίσει και κάνει τις απράιτητες τροποποιήσεις έτσι ώστε τα κείμενα που συλλέχθηκαν να είναι έτοιμα για αποθήκευση.
Η συνάρτηση δέχεται ως παράμετρο την λίστα με τα τις θεματικές και επιστρέφει μια λίστα από έγγραφα.

In [15]:
def scrape_wikipedia_articles(topics):
    articles = []
    
    for topic in topics:
        url = BASE_URL + topic
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Τίτλος άρθρου
            title = soup.find('h1').text
            
            # Περιεχόμενο άρθρου (μόνο από την κύρια παράγραφο)
            paragraphs = soup.find_all('p')
            content = ' '.join([para.text for para in paragraphs if para.text.strip()])
            
            articles.append({
                "title": title,
                "url": url,
                "content": content
            })
        else:
            print(f"Failed to fetch {url}")
    
    return articles

Στη συνέχεια εκτελούμε τη συνάρτηση

In [16]:
# Εκτέλεση
articles = scrape_wikipedia_articles(TOPICS)

Και αποθηκεύουμε τα λίστα με τα έγγραφα που συλλέχθηκαν στο αρχέιο wikipedia_articles.json εμφανίζοντας μήνυμα επιτυχόυς ολοκλήρωσης.

In [17]:
# Αποθήκευση σε JSON
with open("wikipedia_articles.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

print(f"{len(articles)} articles saved in 'wikipedia_articles.json'.")

10 articles saved in 'wikipedia_articles.json'.


## Βήμα 2 - Προεπεξεργασία κειμένου

Για να καθαρίσουμε το κείμενο που έχουμε συλλέξει θα εφαρμόσουμε της παρακάτω τεχνικές με την εξής σειρά:
•	Lowercasing
•	Αφαίρεση ειδικών χαρακτήρων
•	Tokenization
•	Stop-Word Removal
•	Lemmatization
Ο κώδικας περιέχεται στο αρχέιο data_cleaner.py

Κάνουμε import της απραίτητες βιβλιοθήκες και κατεβάζουμε τους απαραίτητους πότους απο το NLTK.

In [18]:
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download

# Κατεβάζουμε τους απαραίτητους πόρους από το NLTK
download('punkt')  # Για tokenization
download('stopwords')  # Για stop-word removal
download('wordnet')  # Για lemmatization
download('omw-1.4')  # Βοηθητικά δεδομένα για lemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mak14\anaconda3\lib\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mak14\anaconda3\lib\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mak14\anaconda3\lib\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mak14\anaconda3\lib\nltk_data...


True

Κάνουμε τις απαράιτητες αρχικοποιήσεις και ανοίγουμε και διαβάζουμε το αρχείο με τα έγγραφα από το προηγούμενο βήμα.

In [19]:
# Αρχικοποιήσεις
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Διαβάζουμε το dataset
with open("wikipedia_articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

Ορίζουμε τη συνάρτηση preprocess_text, η οποία κάνει όλα τα βήματα που αναφέραμε παραπάνω για τον καθαρισμό του κειμένου.

In [20]:
# Συνάρτηση για προεπεξεργασία
def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()
    
    # 2. Αφαίρεση ειδικών χαρακτήρων (κρατάμε μόνο γράμματα και κενά)
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. Tokenization
    tokens = word_tokenize(text)
    
    # 4. Stop-word removal
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Επιστροφή των καθαρισμένων tokens ως ενιαίο κείμενο
    return ' '.join(lemmatized_tokens)

Επεξεργαζόμστε τα άρθρα έτσι ώστε να είναι έτοιμα για αποθύκευση.

In [21]:
# Επεξεργασία των άρθρων
processed_articles = []
for article in articles:
    cleaned_content = preprocess_text(article["content"])
    processed_articles.append({
        "title": article["title"],
        "url": article["url"],
        "cleaned_content": cleaned_content
    })

Και τελικά τα αποθηκεύουμε στο αρχείο processed_wikipedia_articles.json και εμφανίζουμε ανάλογο μήνυμα.

In [22]:
# Αποθήκευση του "καθαρισμένου" dataset
with open("processed_wikipedia_articles.json", "w", encoding="utf-8") as f:
    json.dump(processed_articles, f, ensure_ascii=False, indent=4)

print(f"Preprocessing completed! Cleaned data saved in 'processed_wikipedia_articles.json'.")


Preprocessing completed! Cleaned data saved in 'processed_wikipedia_articles.json'.


## Βήμα 3 - Ευρετήριο

Για να υλοποιήσουμε το ανεστραμμένο ευρετήριο χρησιμοποιούμε το καθαρισμένο κείμενο από το προηγούμενο βήμα και δημιουργούμε μια λίστα λέξεων (tokens) η οποία με τη χρήση μιας δομής λεξικού defaultdict τα αποθηκεύει κατάλληλα και το δημιουργεί. Ο κώδικας περιέχεται στο αρχείο inverted_index.py

Αρχικά κάνουμε import τις απραίτητες βιβλιοθήκες.

In [23]:
import json
from collections import defaultdict

Στη συνέχεια φορτώνουμε το καθαρισμένο κείμενο απο το αρχείο που δημιουργήσαμε στο πορηγούμενω βήμα και ορίζουμε το ανεστραμένο ευρετήριο.

In [24]:
# Φόρτωση των καθαρισμένων άρθρων από το προηγούμενο βήμα
with open("processed_wikipedia_articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

# Δημιουργία ανεστραμμένου ευρετηρίου
inverted_index = defaultdict(list)

Κάνουμε τις κατάλληλες τροποποίησεις στο κείμενο για να κατασκευάζσουμε σωστά το ανεστραμένο ευρετήριο.

In [26]:
#Κατασκευή του ανεστραμμένου ευρετηρίου
for doc_id, article in enumerate(articles):
    # Διαχωρισμός των λέξεων από το καθαρισμένο περιεχόμενο
    tokens = article["cleaned_content"].split()
    # Για κάθε μοναδική λέξη στο έγγραφο
    for token in set(tokens):  # Χρησιμοποιούμε set για να μην αποθηκεύσουμε διπλές εμφανίσεις
        inverted_index[token].append(doc_id)

Αποθηκέυουμε το ευρετήριο στο αρχείο inverted_index.py και εμφανίζουμε ανάλογο μήνυμα.

In [27]:
# Αποθήκευση του ανεστραμμένου ευρετηρίου
with open("inverted_index.json", "w", encoding="utf-8") as f:
    json.dump(inverted_index, f, ensure_ascii=False, indent=4)

print("Inverted index created and save in 'inverted_index.json'.")

Inverted index created and save in 'inverted_index.json'.


## Βήμα 4α - Μηχανή αναζήτησης: Επεξεργασία ερωτήματος

Για να υλοποιήσουμε τον λογικό επεξεργαστή ερωτημάτων που χειρίζεται Boolean ερωτήματα, αρχικά πρέπει να αναλύσουμε το ερώτημα του χρήστη. Αυτό το κατάφερνουμε κατασκεύζοντας μια συνάρτηση που αναλύει το ερώτημα σε tokens και στη συνέχεια ελέγχουμε σε ποια έγργαφα υπάρχουν τα tokens αυτά μέσο του ευρετηρίου που δημιουργήσαμε στο προηγούμενο βήμα. Όλα αυτά δημιουργόντας μια διεπαφή που επιτρέπει στο χρήστη να αλληλεπιδρά με τη μηχανή αναζήτησης.

Κάνουμε import τις απραίτητες βιβλιοθήκες.

In [28]:
import json
import re

Ορίζουμε τη συνάρτηση προεπεξεργασίας των ερωτημάτων.

In [29]:
# Προεπεξεργασία ερωτήματος
def preprocess_query(query):

    query_tokens = re.findall(r'\w+', query.lower())
    return query_tokens

Ορίζουμε τη συνάρτηση αξιολόγισης των ερωτημάτων.

In [30]:
# Αξιολόγηση ερωτήματος
def evaluate_query(query, inverted_index):
    
    tokens = preprocess_query(query)
    result_set = set()
    current_operator = "OR"  # Default operator

    for token in tokens:
        if token in {"and", "or", "not"}:
            current_operator = token.upper()
        else:
            token_docs = set(inverted_index.get(token, []))
            if current_operator == "OR":
                result_set |= token_docs
            elif current_operator == "AND":
                result_set &= token_docs
            elif current_operator == "NOT":
                result_set -= token_docs

    return result_set

Ορίζουμε τη συνάρτηση που υλοποιεί τη διεπαφή με την οποία αλληλεπιδρά ο χρήστης, η οποία φορτώνει τα αρχεία του ευρετηρίου και των εγγράφων απο τα πορηγούμενα ερωτήματα και εμφανίζει στο χρήστη τα αποτελέσματα της αναζήτησης σε κατάλληλη μορφή.

In [32]:
# Διεπαφή χρήστη
def search_engine_cli(inverted_index_path, articles_path):
    # Φόρτωση δεδομένων
    with open(inverted_index_path, "r", encoding="utf-8") as f:
        inverted_index = json.load(f)

    with open(articles_path, "r", encoding="utf-8") as f:
        articles = json.load(f)

    print("Welcome to the Search Engine")
    print("Insert your query or type 'exit' for termination:")

    while True:
        query = input("\nQuery: ")
        if query.lower() == "exit":
            print("Thanks for using the Search Enigne! Terminating...")
            break

        matching_docs = evaluate_query(query, inverted_index)

        if matching_docs:
            print("\nThe suitable files are:")
            for doc_id in matching_docs:
                doc_content = articles[doc_id - 1]["content"]
                print(f"\n[File {doc_id}]")
                print(doc_content[:200] + "...")  # Εμφανίζει τα πρώτα 200 χαρακτήρες
        else:
            print("\nNo relatable files found. Please try again:")

Τέλος, εκτελούμε τις συναρτήσεις.

In [35]:
# Κύρια εκτέλεση
if __name__ == "__main__":
    # Αρχεία JSON που δημιουργήθηκαν στα προηγούμενα βήματα
    inverted_index_path = "inverted_index.json"       # Το ανεστραμμένο ευρετήριο
    articles_path = "wikipedia_articles.json"         # Τα άρθρα της Wikipedia

    # Εκτέλεση της διεπαφής
    search_engine_cli(inverted_index_path, articles_path)

Welcome to the Search Engine
Insert your query or type 'exit' for termination:

Query: information and intelligence

The suitable files are:

[File 1]
Information retrieval (IR) in computing and information science is the task of identifying and retrieving information system resources that are relevant to an information need.  The information need c...

[File 2]
Natural language processing (NLP) is a subfield of computer science and especially artificial intelligence. It is primarily concerned with providing computers with the ability to process data encoded ...

[File 3]
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies metho...

[File 4]
Data mining is the process of extracting and discovering patterns in large data sets involving methods at the intersection of machine learning, statistics, and database systems.[1] Data mining is an i...

[F

Παρατηρούμε ότι με τη δίνοντας ως ερώτημα το "information and intelligence", η μηχανή μας επιστρέφει όλα τα άρθα που περιλαμβάνουν και τους δύο όρους, ενώ όταν δίνουμε το ερώτημα "information not intelligence", μας επιστρέφει τα άρθρα που περιέχονυ μόνο τον όρο "information" και όχι αυτά που περιέχονυ τον όρο "intelligence". 

## Βήμα 4β - Μηχανή Αναζήτησης: Κατάταξη αποτελεσμάτων

Επεκτείνοντας τον κώδικα του ερωτήματος 4. α), ενσωματώνουμε στη μηχανή αναζήτησης τις λειτουργίες κατάταξης των αποτελεσμάτων με χρήση των τεχνικών TF-IDF και ΒΜ25, καθώς επίσης και την εμφάνιση των αποτελεσμάτων αυτών των λειτουργειών σε φιλική προς το χρήστη μορφή. 

Αρχικά κάνουμε import τις απαράιτητες βιβλιοθήκες.

In [ ]:
from skleaρn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import json

Υλοποιούμε μια συνάρτηση για κάθε είδος λειτουργίας. Ξεκινάμε με την λειτουργία της Boolean αναζήτησης η οποία βασίζεται στην συνάρτηση του προηούμενου ερωτήματος.

In [36]:
# Boolean Αναζήτηση
def boolean_search(query, inverted_index, documents):
    
    terms = query.split()
    result_set = set(range(len(documents)))  # Ξεκινάμε με όλα τα έγγραφα
    operation = "OR"  # Default operator
    current_set = set()

    for term in terms:
        if term.upper() in ["AND", "OR", "NOT"]:
            operation = term.upper()
        else:
            matching_docs = set(inverted_index.get(term, []))
            if operation == "AND":
                current_set = current_set & matching_docs if current_set else matching_docs
            elif operation == "OR":
                current_set = current_set | matching_docs
            elif operation == "NOT":
                current_set = current_set - matching_docs
    return current_set

Μετά υλοποιούμε την συνάρτηση της TF-IDF κατάταξης για την οποία χρησιμοποιούμε τον έτοιμο αλγόριθμο που την εφαρμόζει.

In [37]:
# TF-IDF Κατάταξη (Vector Space Model)
def tfidf_ranking(query, documents):
    
    vectorizer = TfidfVectorizer()
    doc_vectors = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    scores = cosine_similarity(query_vector, doc_vectors).flatten()
    return sorted(enumerate(scores, start=0), key=lambda x: x[1], reverse=True)

Υλοποιούμε και την συνάρτηση της ΒΜ25 κατάταξης για την οποία χρησιμπούμε τον έτοιμο αλγόριθμο που την εφαρμόζει.

In [38]:
# BM25 Κατάταξη
def bm25_ranking(query, documents):
    
    tokenized_docs = [doc.split() for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)
    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)
    return sorted(enumerate(scores, start=0), key=lambda x: x[1], reverse=True)

Τέλος, υλοποιούμε την διεπαφή με την οποία θα αλληλεπιδρά ο χρήστης και του δίνει την δυνατότητα επιλογής λειτουργίας και εμφανίζει τα αποτελέσματα. Και εκτελούμε το πρόγραμμα.

In [6]:
# Διεπαφή αναζήτησης με επιλογή τεχνικής
def search_engine(query, algorithm="Boolean", articles_path="wikipedia_articles.json", index_path="inverted_index.json"):
   
    # Φόρτωση άρθρων
    with open(articles_path, "r", encoding="utf-8") as f:
        articles = json.load(f)
    documents = [article["content"] for article in articles]
    titles = [article["title"] for article in articles]

    # Φόρτωση ανεστραμμένου ευρετηρίου
    with open(index_path, "r", encoding="utf-8") as f:
        inverted_index = json.load(f)

    if algorithm == "Boolean":
        boolean_results = boolean_search(query, inverted_index, documents)
        if boolean_results:
            print("\nResults of Boolean Search:")
            for doc_id in boolean_results:
                print(f"- {titles[doc_id]}")
        else:
            print("No relatable files found.")

    elif algorithm == "TF-IDF":
        ranked_results = tfidf_ranking(query, documents)
        print("\nSearch Results (TF-IDF):")
        for doc_id, score in ranked_results[:10]:  # Top 10 αποτελέσματα
            if score > 0:
                print(f"- {titles[doc_id]} (Score: {score:.4f})")

    elif algorithm == "BM25":
        ranked_results = bm25_ranking(query, documents)
        print("\nSearch Results (BM25):")
        for doc_id, score in ranked_results[:10]:  # Top 10 αποτελέσματα
            if score > 0:
                print(f"- {titles[doc_id]} (Score: {score:.4f})")

    else:
        print("Unkown algorithm.")

# Εκτέλεση μηχανής αναζήτησης
if __name__ == "__main__":
    while True:
        print("\nWelcome to the Search Engine.")
        print("1. Boolean Search")
        print("2. Rank Results (TF-IDF)")
        print("3. Rank Results (BM25)")
        print("4. Exit")
        choice = input("Choose your operation: ")

        if choice == "1":
            query = input("Insert the Boolean Query: ")
            search_engine(query, algorithm="Boolean")
        elif choice == "2":
            query = input("Insert your Query: ")
            search_engine(query, algorithm="TF-IDF")
        elif choice == "3":
            query = input("Insert your Query: ")
            search_engine(query, algorithm="BM25")
        elif choice == "4":
            print("Exiting the Search Engine.")
            break
        else:
            print("Not valid option. Please try again.")


Welcome to the Search Engine.
1. Boolean Search
2. Rank Results (TF-IDF)
3. Rank Results (BM25)
4. Exit
Choose your operation: 4
Exiting the Search Engine.


Παρακάτω δίνετε ολίκληρως ο κώδικας της μηχανής αναζήτησης για να μπορέσει να τρέξει σωστά στο Jupyter Notebook.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import json

# Boolean Αναζήτηση
def boolean_search(query, inverted_index, documents):
    
    terms = query.split()
    result_set = set(range(len(documents)))  # Ξεκινάμε με όλα τα έγγραφα
    operation = "OR"  # Default operator
    current_set = set()

    for term in terms:
        if term.upper() in ["AND", "OR", "NOT"]:
            operation = term.upper()
        else:
            matching_docs = set(inverted_index.get(term, []))
            if operation == "AND":
                current_set = current_set & matching_docs if current_set else matching_docs
            elif operation == "OR":
                current_set = current_set | matching_docs
            elif operation == "NOT":
                current_set = current_set - matching_docs
    return current_set

# TF-IDF Κατάταξη (Vector Space Model)
def tfidf_ranking(query, documents):
    
    vectorizer = TfidfVectorizer()
    doc_vectors = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    scores = cosine_similarity(query_vector, doc_vectors).flatten()
    return sorted(enumerate(scores, start=0), key=lambda x: x[1], reverse=True)

# BM25 Κατάταξη
def bm25_ranking(query, documents):
    
    tokenized_docs = [doc.split() for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)
    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)
    return sorted(enumerate(scores, start=0), key=lambda x: x[1], reverse=True)

# Διεπαφή αναζήτησης με επιλογή τεχνικής
def search_engine(query, algorithm="Boolean", articles_path="wikipedia_articles.json", index_path="inverted_index.json"):
   
    # Φόρτωση άρθρων
    with open(articles_path, "r", encoding="utf-8") as f:
        articles = json.load(f)
    documents = [article["content"] for article in articles]
    titles = [article["title"] for article in articles]

    # Φόρτωση ανεστραμμένου ευρετηρίου
    with open(index_path, "r", encoding="utf-8") as f:
        inverted_index = json.load(f)

    if algorithm == "Boolean":
        boolean_results = boolean_search(query, inverted_index, documents)
        if boolean_results:
            print("\nResults of Boolean Search:")
            for doc_id in boolean_results:
                print(f"- {titles[doc_id]}")
        else:
            print("No relatable files found.")

    elif algorithm == "TF-IDF":
        ranked_results = tfidf_ranking(query, documents)
        print("\nSearch Results (TF-IDF):")
        for doc_id, score in ranked_results[:10]:  # Top 10 αποτελέσματα
            if score > 0:
                print(f"- {titles[doc_id]} (Score: {score:.4f})")

    elif algorithm == "BM25":
        ranked_results = bm25_ranking(query, documents)
        print("\nSearch Results (BM25):")
        for doc_id, score in ranked_results[:10]:  # Top 10 αποτελέσματα
            if score > 0:
                print(f"- {titles[doc_id]} (Score: {score:.4f})")

    else:
        print("Unkown algorithm.")

# Εκτέλεση μηχανής αναζήτησης
if __name__ == "__main__":
    while True:
        print("\nWelcome to the Search Engine.")
        print("1. Boolean Search")
        print("2. Rank Results (TF-IDF)")
        print("3. Rank Results (BM25)")
        print("4. Exit")
        choice = input("Choose your operation: ")

        if choice == "1":
            query = input("Insert the Boolean Query: ")
            search_engine(query, algorithm="Boolean")
        elif choice == "2":
            query = input("Insert your Query: ")
            search_engine(query, algorithm="TF-IDF")
        elif choice == "3":
            query = input("Insert your Query: ")
            search_engine(query, algorithm="BM25")
        elif choice == "4":
            print("Exiting the Search Engine.")
            break
        else:
            print("Not valid option. Please try again.")


Welcome to the Search Engine.
1. Boolean Search
2. Rank Results (TF-IDF)
3. Rank Results (BM25)
4. Exit
Choose your operation: 1
Insert the Boolean Query: information and intelligence

Results of Boolean Search:
- Natural language processing
- Artificial intelligence
- Data mining
- Machine learning
- Deep learning
- Computer vision
- Neural network
- Big data

Welcome to the Search Engine.
1. Boolean Search
2. Rank Results (TF-IDF)
3. Rank Results (BM25)
4. Exit
Choose your operation: 1
Insert the Boolean Query: information not intelligence

Results of Boolean Search:
- Information retrieval
- Data science

Welcome to the Search Engine.
1. Boolean Search
2. Rank Results (TF-IDF)
3. Rank Results (BM25)
4. Exit
Choose your operation: 2
Insert your Query: use

Search Results (TF-IDF):
- Artificial intelligence (Score: 0.0323)
- Data mining (Score: 0.0269)
- Big data (Score: 0.0260)
- Data science (Score: 0.0247)
- Machine learning (Score: 0.0173)
- Deep learning (Score: 0.0145)
- Comput

Εκτελόυμε μία μία τις λειτουργίες της μηχανής και παρατηρούμε ότι η λειτουργία της Boolean αναζήτησης λειτουργεί όπως και στην προηγούμενη έκδωση της μηαχνής, αλλά αυτή τη φορά εμφανίζονται οι τίτλοι των εγγράφων που αντιστοιχούν στις προϋποθέσεις που ορίζουν τα queries. Για να ελέγξουμε τις λειτουργίες των TF-IDF και BM25 κατατάξεων δίνουμε ώς όρο στη τη λέξη "use" και παρατηρούμε ότι η μηαχανή μας επιστρέφει τις κατατάξεις των εγγράφων ανάλογα με το πόσο βρίσκεται ο όρος που δώσαμε στο καθένα και σύμφωνα με τα κρητίρια της κάθε μεθόδου. Για παράδειγμα, βλέπουμε ότι ο όρος "use" έχει πιο μεγάλο score στο έγγραφο "artificial intelligence" και είναι πιο ψηλά στην κατάταξη χρησιμοποιώντας την TF-IDF σε σχέση με την BM25.

## Βήμα 5 - Αξιολόγηση συστήματος

Για να υλοποιήσουμε την αξιολόγηση τους συστήματος χρησιμοποιούμε τις συναρτήσεις από πριν που υπολογίζουν τις κατατάξεις και την Boolean αναζήτηση και με τη χρήση μερικών queries αξιολόγησης στα οποία εφαρμόζουμε όλες τις παραπάνω λειτουργίες, υπολογίζουμε τη μέση ακρίβεια, τη μέση ανάκληση, το μέσο F1-score και τη μέση ακρίβεια μέσης θέσης.

Ξεκινάμε κάνοντας import τις απραίτητες βιβλιοθήκες.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from sklearn.metrics import precision_score, recall_score, f1_score
import json
import numpy as np

Στη συνέχεια υλοποιούμε μία συνάρτηση για κάθε μέθοδο αξιολόγησης χρησιμοποιώντας τους έτοιμους αλγόριθμους από τις βιβλιοθήκες.

In [9]:
# TF-IDF Κατάταξη
def tfidf_ranking(query, documents):
    vectorizer = TfidfVectorizer()
    doc_vectors = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    scores = cosine_similarity(query_vector, doc_vectors).flatten()
    return sorted(enumerate(scores, start=0), key=lambda x: x[1], reverse=True)

# BM25 Κατάταξη
def bm25_ranking(query, documents):
    tokenized_docs = [doc.split() for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)
    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)
    return sorted(enumerate(scores, start=0), key=lambda x: x[1], reverse=True)

# Boolean Αναζήτηση
def boolean_search(query, inverted_index):
    terms = query.lower().split()
    matching_docs = set(inverted_index.get(terms[0], []))
    for term in terms[1:]:
        if term in inverted_index:
            matching_docs = matching_docs.intersection(inverted_index[term])
    return sorted(matching_docs)

Φορτώνουμε το inverted index από το βήμα 3, στο οποίο θα ψάξει η μηχανή αναζήτησης και δημιουργούμε μια λίστα με queries με βάση τα οποία θα γίνει η αξιολόγηση του συστήματος.

In [11]:
# Φόρτωση Inverted Index
def load_inverted_index(path="inverted_index.json"):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# Αξιολόγηση
test_queries = [
    {"query": "machine learning", "relevant_docs": {0, 2}},
    {"query": "data science", "relevant_docs": {1, 3}},
    {"query": "natural language processing", "relevant_docs": {4}},
    {"query": "artificial intelligence", "relevant_docs": {0, 4, 5}},
    {"query": "deep learning", "relevant_docs": {2, 5}},
]

Τέλος δημιουργούμε την κ΄θρια συνάρτηση του προγράμματος ή οποία καλεί της υπόλοιπες και πραγαμτοποιεί την αξιολόγηση του συστήματος και εκτελούμε το πρόγραμμα αξιολόγησης καλώντας την κάθε φορά με μια από τις τρείς διαφορετικές μεθόδους.

In [13]:
def evaluate_search_engine(algorithm, articles_path="wikipedia_articles.json", inverted_index_path="inverted_index.json"):
    with open(articles_path, "r", encoding="utf-8") as f:
        articles = json.load(f)

    documents = [article["content"] for article in articles]
    all_precisions = []
    all_recalls = []
    all_f1_scores = []
    average_precisions = []

    if algorithm == "Boolean":
        inverted_index = load_inverted_index(inverted_index_path)

    for test in test_queries:
        query = test["query"]
        relevant_docs = test["relevant_docs"]

        if algorithm == "Boolean":
            retrieved_docs = boolean_search(query, inverted_index)
        elif algorithm == "TF-IDF":
            ranked_results = tfidf_ranking(query, documents)
            retrieved_docs = [doc_id for doc_id, score in ranked_results if score > 0]
        elif algorithm == "BM25":
            ranked_results = bm25_ranking(query, documents)
            retrieved_docs = [doc_id for doc_id, score in ranked_results if score > 0]
        else:
            raise ValueError("Unknown algorithm!")

        y_true = [1 if i in relevant_docs else 0 for i in range(len(documents))]
        y_pred = [1 if i in retrieved_docs else 0 for i in range(len(documents))]

        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1_scores.append(f1)

        ap = 0
        hits = 0
        for rank, doc_id in enumerate(retrieved_docs, start=1):
            if doc_id in relevant_docs:
                hits += 1
                ap += hits / rank
        average_precisions.append(ap / len(relevant_docs) if relevant_docs else 0)

    mean_precision = np.mean(all_precisions)
    mean_recall = np.mean(all_recalls)
    mean_f1 = np.mean(all_f1_scores)
    mean_ap = np.mean(average_precisions)

    print(f"\nEvaluation of Search Engine (Algorithm: {algorithm}):")
    print(f"- Average Precision: {mean_precision:.4f}")
    print(f"- Average Recall: {mean_recall:.4f}")
    print(f"- Average F1-score: {mean_f1:.4f}")
    print(f"- Mean Average Precision (MAP): {mean_ap:.4f}")

# Εκτέλεση Αξιολόγησης
print("Start of Evaluation...\n")
evaluate_search_engine("TF-IDF")
evaluate_search_engine("BM25")
evaluate_search_engine("Boolean")

Start of Evaluation...


Evaluation of Search Engine (Algorithm: TF-IDF):
- Average Precision: 0.2039
- Average Recall: 0.9333
- Average F1-score: 0.3293
- Mean Average Precision (MAP): 0.3401

Evaluation of Search Engine (Algorithm: BM25):
- Average Precision: 0.2039
- Average Recall: 0.9333
- Average F1-score: 0.3293
- Mean Average Precision (MAP): 0.3922

Evaluation of Search Engine (Algorithm: Boolean):
- Average Precision: 0.2289
- Average Recall: 0.8333
- Average F1-score: 0.3558
- Mean Average Precision (MAP): 0.3600


Παίρνουμε τα αποτελέσματα τις αξιολόγησης της μηχανής αναζήτησης κάθε μεθόδου και για κάθε χαρακτηριστικό με βάση τα queries που ορίσαμε μέσα στο πρόγραμμα.